 지난 `Part 02`에서는 Federated Learning을 사용하여 `toy model`을 학습했습니다. 모델에서 `.send ()` 및 `.get ()`을 호출하여 훈련 데이터의 위치로 전송하고 업데이트 한 다음 다시 가져옵니다. 

 그러나 이 예의 끝에서 우리는 **사람들의 프라이버시를 보호하기 위해 조금 더 해야할 것**이 있었다. 즉, `.get ()`을 호출하기 전에 **average the gradients(그라디언트를 평균화)**하려고 합니다. 그렇게하면 누구의 정확한 gradient 볼 수 없게 됩니다 (따라서 프라이버시를 보호하는 것이 좋습니다 !!!).

그러나 이렇게하려면 몇 가지 작업이 더 필요합니다.

- 포인터를 사용하여 텐서를 다른 작업자에게 직접 보내자
- 이번 예제와 향후 몇 가지를 모두 사용하는 데 도움이되는 몇 가지 고급 텐서 작업에 대해 알아보자.

In [1]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

### Section 3.1 - Pointers to Pointers

As you know, PointerTensor objects feel just like normal tensors. In fact, they are so much like tensors that we can even have pointers to the pointers. Check it out!

In [2]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [12]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

In [13]:
# this send the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer

In [14]:
x_ptr

(Wrapper)>[PointerTensor | me:19521867648 -> bob:3750187241]

In [15]:
# now we can SEND THE POINTER to alice
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

(Wrapper)>[PointerTensor | me:58735176500 -> alice:19521867648]

### What happened?

앞에서는 `x`라는 텐서를 생성하여 로컬 머신 `x_ptr`에 포인터를 생성하여 `Bob`에게 보냈다.

그런 다음 `x_ptr.send(alice)`를 호출하여 `Alice`에게 포인터를 보냈습니다.

이것은 **데이터를 이동시키지 않았습니다! 대신, 포인터를 데이터로 옮겼습니다 !!**

In [16]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{3750187241: tensor([1, 2, 3, 4])}

In [17]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

{19521867648: (Wrapper)>[PointerTensor | alice:19521867648 -> bob:3750187241]}

In [18]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

(Wrapper)>[PointerTensor | me:19521867648 -> bob:3750187241]

In [19]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

tensor([1, 2, 3, 4])

### Arithmetic on Pointer -> Pointer -> Data Object

And just like with normal pointers, we can perform arbitrary PyTorch operations across these tensors.

In [20]:
bob._objects

{}

In [22]:
alice._objects

{}

In [23]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [25]:
bob._objects

{56156411406: tensor([1, 2, 3, 4, 5]),
 53548862386: tensor([ 2,  4,  6,  8, 10])}

In [26]:
alice._objects

{16309706200: (Wrapper)>[PointerTensor | alice:16309706200 -> bob:56156411406],
 45027365156: (Wrapper)>[PointerTensor | alice:45027365156 -> bob:53548862386]}

In [27]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [28]:
bob._objects

{56156411406: tensor([1, 2, 3, 4, 5])}

In [29]:
alice._objects

{16309706200: (Wrapper)>[PointerTensor | alice:16309706200 -> bob:56156411406]}

In [30]:
p2p2x.get().get()

tensor([1, 2, 3, 4, 5])

In [31]:
bob._objects

{}

In [32]:
alice._objects

{}

### Section 3.2 - Pointer Chain Operations

`Section 3.1`에서 `.send()` 또는 `.get()` 작업을 호출 할 때마다 로컬 시스템의 텐서에서 직접 해당 작업을 호출했습니다. 그러나 `pointer chain`이 있는 경우 chain의 마지막 pointer에서 `.get()` 또는 `.send()`와 같은 작업을 호출하려는 경우가 있습니다.  
(예 : 한 작업자에서 다른 작업자에게 직접 데이터 보내기). 이를 위해 `이 개인 정보 보호 작업을 위해 특별히 설계된 기능`을 사용하려고합니다.

These operations are:

- `my_pointer2pointer.move(another_worker)`

In [33]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [35]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {49849131043: tensor([1, 2, 3, 4, 5])}
alice: {}


In [36]:
x = x.move(alice)

In [37]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {}
alice: {49849131043: tensor([1, 2, 3, 4, 5])}


In [38]:
x

(Wrapper)>[PointerTensor | me:76556006464 -> alice:49849131043]

**Excellent! Now we're equiped with the tools to perform remote gradient averaging using a trusted aggregator!**